In [1]:
import ModelCreatorWord as md

authorList = [23, 59, 88] 
doc_id = 227
chunk_size = 1000
dimensions = 100
embedfile = 'glove.6B.' + str(dimensions) + 'd.txt'

Using Theano backend.
Using gpu device 0: GeForce GTX 950 (CNMeM is disabled, cuDNN 5005)


In [ ]:
# embeddings_index = md.readVectorData(embedfile, GLOVE_DIR='../../glove/')
# (testX, textY) = md.loadDocData(authorList, doc_id, chunk_size = chunk_size)

In [ ]:
parameters = {
    'dropout': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'samples': [100, 150, 200, 250, 300, 400, 600, 1000, 1500],
    'dimensions': [100, 200], # [50, 100, 200, 300],
    # 'dimensions': [50, 100, 200, 300],
    'iterations': [20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
    # 'iterations': [20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
}
# resultWord = {
#     'dropout': [0, 0, 0, 0, 0, 0],
#     'samples': [0, 0, 0, 0, 0, 0],
#     'dimensions': [0, 0, 0, 0],
#     'iterations': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
# }
resultWord = {
    'Parameters': 'Accuracy'
}

level = 'word'

# samplelist = parameters['samples']

samples = 100

dimensionslist = parameters['dimensions']

for dimensions in dimensionslist:
    
    embeddings_index = md.readVectorData(embedfile, GLOVE_DIR='../../glove/')
    
    (testX, textY) = md.loadDocData(authorList, doc_id, chunk_size = chunk_size)
    
    (texts, labels, labels_index) = md.loadAuthData(authorList, doc_id, chunk_size = chunk_size, 
                                                    samples = samples)
    
    (trainX, trainY, valX, valY) = md.preProcessTrainVal(texts, labels, chunk_size = chunk_size)
    
    embedding_matrix = md.prepareEmbeddingMatrix(embeddings_index, EMBEDDING_DIM = dimensions)
    
    (testX, textY) = md.preProcessTest(testX, labels_index, textY, chunk_size = chunk_size)
    
    droplist = parameters['dropout']
    
    for dropout in droplist:
        
        iterlist = parameters['iterations']
        
        for iterations in iterlist:
            
            testpara = (('Dimensions:%4s     Samples:%4s     Dropout:%4s     Iterations:%4s') % (str(dimensions),
                                                                                                 samples,
                                                                                                 dropout,
                                                                                                 iterations))
            print("%s" % (str(testpara)))
            
            model = md.compileModel(len(labels_index), embedding_matrix, chunk_size = chunk_size, 
                                    DROP_OUT = dropout, EMBEDDING_DIM = dimensions)
            
            (model, history) = md.fitModel(model, trainX, trainY, valX, valY, nb_epoch = iterations)
            
            (predYList, predY) = md.predictModel(model, testX)

            del model
            
            print("%s \nPred: %5s" % (str(testpara), str(predY[0])))
            
            resultWord[testpara] = predY[0]
            
            from IPython.display import clear_output
            
            clear_output()

Dimensions: 100     Samples: 100     Dropout: 0.4     Iterations:  80
Done compiling.
Train on 240 samples, validate on 60 samples
Epoch 1/80
240/240 [==============================] - 1s - loss: 1.1037 - acc: 0.3292 - val_loss: 1.0963 - val_acc: 0.3833
Epoch 2/80
240/240 [==============================] - 1s - loss: 1.1076 - acc: 0.2917 - val_loss: 1.0967 - val_acc: 0.4000
Epoch 3/80
240/240 [==============================] - 1s - loss: 1.0949 - acc: 0.3667 - val_loss: 1.0971 - val_acc: 0.3833
Epoch 4/80
200/240 [========================>.....] - ETA: 0s - loss: 1.0985 - acc: 0.3150

In [ ]:
# print resultWord

In [ ]:
resultWord.pop('Parameters')

In [ ]:
predmaxkey = max(resultWord, key = resultWord.get)
predmaxvalue = resultWord[predmaxkey]
print predmaxkey
print predmaxvalue

In [ ]:
# Dimensions: 100     Samples: 150     Dropout: 0.1     Iterations: 140
# 0.730941

In [ ]:
# import matplotlib.pyplot as plt
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()